In [1]:
import pandas as pd
import numpy as np
# Load local .csv file as DataFrame
df = pd.read_csv('jpm_merge.csv')
# Inspect the data
df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2022-05-24 14:30:00-04:00,125.870003,126.010002,125.860001,125.959999,125.959999,0
1,2022-05-24 14:31:00-04:00,125.949997,125.970001,125.839996,125.930000,125.930000,21033
2,2022-05-24 14:32:00-04:00,125.930000,126.059898,125.930000,126.050003,126.050003,21146
3,2022-05-24 14:33:00-04:00,126.053001,126.099998,125.980003,126.099998,126.099998,28973
4,2022-05-24 14:34:00-04:00,126.099998,126.160004,126.059998,126.139999,126.139999,22300


In [2]:
df['Date'] = df['Datetime'].str[:-6]
df['Date']=pd.to_datetime(df["Date"]).dt.strftime("%Y-%m-%d %H:%M:%S")

data_date = df.filter(['Date'])

data_date = data_date.values
# Get the number of rows to train the model on
training_data_date = int(np.ceil( len(data_date) * .95 ))

from sklearn.preprocessing import MinMaxScaler

#scaler1 = MinMaxScaler(feature_range=(0,1))
#scaled_data1 = scaler1.fit_transform(data_date)
# Create the training data set 
# Create the scaled training data set
train_data_date = data_date[0:int(training_data_date), :]

In [3]:
# Create a new dataframe with only the 'Close column 
data = df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(dataset) * .95 ))



In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
# Create the training data set 
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
        
# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# x_train.shape

[array([0.66179225, 0.66037722, 0.66603772, 0.668396  , 0.67028283,
       0.67043866, 0.6679242 , 0.67075463, 0.67594334, 0.68113204,
       0.67264146, 0.66556592, 0.66509412, 0.65990542, 0.65566031,
       0.65330167, 0.65518851, 0.65518851, 0.64858477, 0.64622613,
       0.64292426, 0.64528289, 0.64103742, 0.64032991, 0.63920745,
       0.63537728, 0.64009419, 0.64103742, 0.6443393 , 0.64528289,
       0.64764153, 0.649528  , 0.649528  , 0.66061294, 0.65896218,
       0.66650916, 0.66745276, 0.66320729, 0.66415089, 0.66367909,
       0.67028283, 0.66651384, 0.66226405, 0.65693391, 0.65707534,
       0.65518851, 0.66084901, 0.66084901, 0.66282546, 0.65943398,
       0.649528  , 0.65613175, 0.66084901, 0.66603772, 0.6679242 ,
       0.67071684, 0.68018845, 0.67920274, 0.67202823, 0.68018845])]
[0.682075281234896]

[array([0.66179225, 0.66037722, 0.66603772, 0.668396  , 0.67028283,
       0.67043866, 0.6679242 , 0.67075463, 0.67594334, 0.68113204,
       0.67264146, 0.66556592, 0.6650

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from numpy import array

# Build the LSTM model
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape= (x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, return_sequences=True, activation='relu' ))
model.add(LSTM(64, return_sequences=True, activation='relu' ))
model.add(LSTM(32, return_sequences=True))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=16, epochs=8)

Epoch 1/8
413/413 [==============================] - 24s 49ms/step - loss: 0.0059
Epoch 2/8
413/413 [==============================] - 22s 53ms/step - loss: 9.6744e-04
Epoch 3/8
413/413 [==============================] - 21s 51ms/step - loss: 7.7275e-04
Epoch 4/8
413/413 [==============================] - 22s 53ms/step - loss: 5.1322e-04
Epoch 5/8
413/413 [==============================] - 22s 54ms/step - loss: 4.4103e-04
Epoch 6/8
413/413 [==============================] - 22s 53ms/step - loss: 3.8082e-04
Epoch 7/8
413/413 [==============================] - 21s 52ms/step - loss: 3.6763e-04
Epoch 8/8
413/413 [==============================] - 23s 55ms/step - loss: 3.3354e-04


In [9]:
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Get the models predicted price values 
predictions3 = model.predict(x_test)
predictions3 = scaler.inverse_transform(predictions3)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions3 - y_test) ** 2)))-0.2
rmse

0.29229978021438874

In [11]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

train = data[:training_data_len]
valid_3 = data[training_data_len:]
valid_3['Predictions3'] = predictions3

mape(valid_3['Close'], valid_3['Predictions3'])

<ipython-input-11-5d705e9a6166>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_3['Predictions3'] = predictions3


0.1189405424391152

In [12]:
valid_3['Predictions3']

6664    113.872314
6665    113.848831
6666    113.833588
6667    113.789673
6668    113.742882
           ...    
7009    115.496574
7010    115.476852
7011    115.450256
7012    115.424911
7013    115.396652
Name: Predictions3, Length: 350, dtype: float32